In [87]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

In [88]:
# Load model and data
with open('../models/baseline_model.pkl', 'rb') as f:
   baseline_model = pickle.load(f)

with open('../test_data.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

print(X_test.shape)

(26000, 315)


In [89]:
with open('../models/unbiased_model.pkl', 'rb') as f:
   unbiased_model = pickle.load(f)

In [90]:
def get_protected_feature_indices():
   protected_indices = {
       'demographic': {
           'gender': [215],
           'age': [216],
           'language': [244,245,246,247]
       },
       'location': {
           'neighborhood': list(range(6,11)),
           'district': list(range(13,22)),
           'rotterdam': [11,12]
       },
       'family': {
           'children': list(range(282,289)),
           'partner': [302,303]
       },
       'health': {
           'physical': [54],
           'mental': [55],
           'medical': [67,68,69]
       },
       'financial': {
           'problems': [53,56],
           'income': [132,133,134]
       }
   }
   return protected_indices

protected_indices = get_protected_feature_indices()
age_index = protected_indices['demographic']['age'][0]
neighborhood_indices = protected_indices['location']['neighborhood']

def get_feature_importances(pipeline_or_model):
    """Helper function to get feature importances whether using a pipeline or direct model"""
    if hasattr(pipeline_or_model, 'named_steps'):  # If it's a pipeline
        model = pipeline_or_model.named_steps['classifier']
        return model.feature_importances_
    else:  # If it's just the model
        return pipeline_or_model.feature_importances_

def test_location_importance(model):
    """
    Test 1: Analyzes how much the model relies on location-related features
    """
    importances = get_feature_importances(model)

    location_importance = {
        'neighborhood': sum(importances[6:11]),
        'district': sum(importances[13:22]),
        'rotterdam': sum(importances[i] for i in [11,12])
    }

    total_location_importance = sum(location_importance.values())
    total_model_importance = sum(importances)

    return {
        'location_importance_ratio': total_location_importance / total_model_importance,
        'location_breakdown': location_importance,
        'interpretation': f"Location features account for {(total_location_importance/total_model_importance)*100:.1f}% of model's decision making"
    }

def test_age_discrimination(model, X_test, y_test):
    """
    Test 2: Checks if model discriminates based on age
    """
    age_values = X_test.iloc[:, age_index]
    age_groups = pd.qcut(age_values, q=4, labels=['youngest', 'young', 'middle', 'oldest'])

    group_metrics = {}
    for group in age_groups.unique():
        mask = age_groups == group
        group_preds = model.predict(X_test[mask])
        group_true = y_test[mask]

        group_metrics[group] = {
            'approval_rate': np.mean(group_preds == 1),
            'accuracy': accuracy_score(group_true, group_preds)
        }

    approval_rates = [metrics['approval_rate'] for metrics in group_metrics.values()]
    max_disparity = max(approval_rates) - min(approval_rates)

    return {
        'age_group_metrics': group_metrics,
        'max_approval_disparity': max_disparity,
        'interpretation': f"Maximum approval rate disparity between age groups: {max_disparity:.2%}"
    }

def test_neighborhood_bias(model, X_test, y_test):
    """
    Test 3: Checks for bias in different neighborhoods
    """
    neighborhood_features = X_test.iloc[:, 6:11]
    neighborhoods = neighborhood_features.idxmax(axis=1)

    neighborhood_metrics = {}
    for neighborhood in neighborhood_features.columns:
        mask = neighborhoods == neighborhood
        if sum(mask) > 0:
            n_preds = model.predict(X_test[mask])
            n_true = y_test[mask]

            neighborhood_metrics[neighborhood] = {
                'approval_rate': np.mean(n_preds == 1),
                'accuracy': accuracy_score(n_true, n_preds),
                'sample_size': sum(mask)
            }

    approval_rates = [metrics['approval_rate'] for metrics in neighborhood_metrics.values()]
    max_disparity = max(approval_rates) - min(approval_rates)

    return {
        'neighborhood_metrics': neighborhood_metrics,
        'max_approval_disparity': max_disparity,
        'interpretation': f"Maximum approval rate disparity between neighborhoods: {max_disparity:.2%}"
    }

def test_model_performance(model, X_test, y_test):
    """
    Test 4: Evaluates overall model performance using multiple metrics
    """
    from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

    predictions = model.predict(X_test)

    metrics = {
        'accuracy': accuracy_score(y_test, predictions),
        'precision': precision_score(y_test, predictions),
        'recall': recall_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions)
    }

    conf_matrix = confusion_matrix(y_test, predictions)
    total_predictions = len(predictions)
    positive_rate = np.mean(predictions == 1)

    return {
        'standard_metrics': metrics,
        'confusion_matrix': conf_matrix,
        'total_samples': total_predictions,
        'positive_prediction_rate': positive_rate,
        'interpretation': (
            f"Model Performance:\n"
            f"Accuracy: {metrics['accuracy']:.2%}\n"
            f"Precision: {metrics['precision']:.2%}\n"
            f"Recall: {metrics['recall']:.2%}\n"
            f"F1 Score: {metrics['f1_score']:.2%}\n"
            f"Overall positive prediction rate: {positive_rate:.2%}"
        )
    }

In [91]:
print("Running tests on baseline model...")
location_results = test_location_importance(baseline_model)
age_results = test_age_discrimination(baseline_model, X_test, y_test)
neighborhood_results = test_neighborhood_bias(baseline_model, X_test, y_test)
performance_results = test_model_performance(baseline_model, X_test, y_test)

print("\nTest 1 - Location Importance:")
print(location_results['interpretation'])
print("Breakdown:", location_results['location_breakdown'])

print("\nTest 2 - Age Discrimination:")
print(age_results['interpretation'])
print("Age group metrics:", age_results['age_group_metrics'])

print("\nTest 3 - Neighborhood Bias:")
print(neighborhood_results['interpretation'])
print("Neighborhood metrics:", neighborhood_results['neighborhood_metrics'])

print("\nTest 4 - Model Performance:")
print(performance_results['interpretation'])

Running tests on baseline model...

Test 1 - Location Importance:
Location features account for 2.3% of model's decision making
Breakdown: {'neighborhood': 0.0008401442749126893, 'district': 0.022226259829788556, 'rotterdam': 0.00028413119654318117}

Test 2 - Age Discrimination:
Maximum approval rate disparity between age groups: 14.60%
Age group metrics: {'youngest': {'approval_rate': 0.15360169491525424, 'accuracy': 0.836410411622276}, 'oldest': {'approval_rate': 0.007565789473684211, 'accuracy': 0.8886513157894737}, 'middle': {'approval_rate': 0.016486104569006125, 'accuracy': 0.9470874548594755}, 'young': {'approval_rate': 0.04205674780354314, 'accuracy': 0.9009073887368573}}

Test 3 - Neighborhood Bias:
Maximum approval rate disparity between neighborhoods: 6.02%
Neighborhood metrics: {'adres_recentste_buurt_groot_ijsselmonde': {'approval_rate': 0.05588235294117647, 'accuracy': 0.8959752321981425, 'sample_size': 12920}, 'adres_recentste_buurt_nieuwe_westen': {'approval_rate': 0.05

In [92]:
print("Running tests on unbiased model...")
location_results = test_location_importance(unbiased_model)
age_results = test_age_discrimination(unbiased_model, X_test, y_test)
neighborhood_results = test_neighborhood_bias(unbiased_model, X_test, y_test)
performance_results = test_model_performance(unbiased_model, X_test, y_test)

print("\nTest 1 - Location Importance:")
print(location_results['interpretation'])
print("Breakdown:", location_results['location_breakdown'])

print("\nTest 2 - Age Discrimination:")
print(age_results['interpretation'])
print("Age group metrics:", age_results['age_group_metrics'])

print("\nTest 3 - Neighborhood Bias:")
print(neighborhood_results['interpretation'])
print("Neighborhood metrics:", neighborhood_results['neighborhood_metrics'])

print("\nTest 4 - Model Performance:")
print(performance_results['interpretation'])

Running tests on unbiased model...

Test 1 - Location Importance:
Location features account for 2.9% of model's decision making
Breakdown: {'neighborhood': 0.0012510763979702952, 'district': 0.02709897173658722, 'rotterdam': 0.0005326023104891282}

Test 2 - Age Discrimination:
Maximum approval rate disparity between age groups: 8.44%
Age group metrics: {'youngest': {'approval_rate': 0.100181598062954, 'accuracy': 0.8020581113801453}, 'oldest': {'approval_rate': 0.015789473684210527, 'accuracy': 0.8965460526315789}, 'middle': {'approval_rate': 0.017899199246349504, 'accuracy': 0.9472444653791804}, 'young': {'approval_rate': 0.04205674780354314, 'accuracy': 0.9011954486533199}}

Test 3 - Neighborhood Bias:
Maximum approval rate disparity between neighborhoods: 4.17%
Neighborhood metrics: {'adres_recentste_buurt_groot_ijsselmonde': {'approval_rate': 0.043575851393188855, 'accuracy': 0.8876934984520124, 'sample_size': 12920}, 'adres_recentste_buurt_nieuwe_westen': {'approval_rate': 0.05882